<a href="https://colab.research.google.com/github/tshigata/LLM_Final/blob/main/llm_jp_unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 最終課題コンペ用 Fine-tuning テンプレート（unsloth）

最終課題コンペにて Fine-tuning を行ないたい方に向けの Fine-tuning コードです。
こちらは L4 を利用できない受講生の方向けにUnslothを用いたものとなっております。
Google Colab の無料版で利用可能な T4 でも動作可能になっています。
環境設定の難易度が高いので、慎重に取り組んでいただければと思います。

In [1]:
import os
from datetime import datetime
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Google Colab の場合は上記の環境構築手順を行なわず、単にこのセルから実行していってください。
!pip uninstall -qqq unsloth -y
!pip install -qqq --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 279.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 342.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 376.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 264.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 370.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 331.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 343.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 369.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 284.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 359.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5

In [3]:
# Google Colab のデフォルトで入っているパッケージをアップグレード（Moriyasu さんありがとうございます）
!pip install -qqq --upgrade torch
!pip install -qqq --upgrade xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 96.9 MB/s eta 0:00:00


In [4]:
# notebookでインタラクティブな表示を可能とする（ただし、うまく動かない場合あり）
# Google Colabでは実行不要
!pip install -qqq ipywidgets --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.1 MB/s eta 0:00:00


In [5]:
# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

## モデルのロード
以下のコードでモデルを読み込みます。  
受講生の方からご指摘頂いたのですが、unslothでgemma2を読み込むと、自動でunslothが作成した非公式モデルがダウンロードされるようです。  
対処方法がわからない受講生はLLM-jp-3のみをご利用ください！  

In [6]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')

In [7]:
wandb_api_key = userdata.get('WANDB_API_KEY')
!wandb login $wandb_api_key

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
import os
os.environ["WANDB_PROJECT"] = "llm-final"  # W&Bプロジェクトの名前を指定
#os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # すべてのモデルチェックポイントをログ

In [9]:
from datetime import datetime  # 日付取得用
from pytz import timezone

# 日本時間を設定
jst = timezone('Asia/Tokyo')

# 現在の日付を取得してフォーマット
today_date = datetime.now(jst).strftime("%m-%d")  # MM-DD形式
today_date_time = datetime.now(jst).strftime("%m%d_%H%M")  # MM-DD形式

model_name = "llm-jp-3-13b"
base_model_id = f"llm-jp/{model_name}"
base_model_id = "tshigata/llm-jp-3-13b-bl-11-30_00_lora" #ベースモデルを最高点のものにする
base_model_id = "tshigata/llm-jp-3-13b-1130_1435_lora"  #ベースモデルをOzaki10000のものにする
base_model_id = "tshigata/llm-jp-3-13b-1130_1934_lora" #ベースモデルをOzaki1+4のものにする
new_model_id = f"{model_name}-{today_date_time}"  # NEWモデル名

# new_model_id = f"llm-jp-3-13b-it-{today_date}"  # 日付付きモデル名
# new_model_id = "llm-jp-3-13b-it" #Fine-Tuningしたモデルにつけたい名前、it: Instruction Tuning

print(f"base model: {base_model_id}")
print(f"new model: {new_model_id}")

base model: tshigata/llm-jp-3-13b-1130_1934_lora
new model: llm-jp-3-13b-1130_2227


In [10]:
import os
from datetime import datetime

# ベースパス
base_path = "/content/drive/MyDrive/LLM/_最終課題"

# 今日の日付をフォーマットしてフォルダ名を生成

date_folder = os.path.join(base_path, today_date_time)

# フォルダの存在を確認し、無ければ作成
if not os.path.exists(date_folder):
    os.makedirs(date_folder)
    print(f"フォルダを作成しました: {date_folder}")
else:
    print(f"フォルダは既に存在します: {date_folder}")

%cd $date_folder
%ls

フォルダを作成しました: /content/drive/MyDrive/LLM/_最終課題/1130_2227
/content/drive/MyDrive/LLM/_最終課題/1130_2227


In [11]:
# llm-jp/llm-jp-3-13bを4bit量子化のqLoRA設定でロード。
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # unslothではRoPEをサポートしているのでコンテキスト長は自由に設定可能
dtype = None # Noneにしておけば自動で設定
load_in_4bit = True # 今回は13Bモデルを扱うためTrue


# FastLanguageModel インスタンスを作成
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_model_id,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    trust_remote_code=True,
)

# SFT用のモデルを用意
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, #16, #8, #32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32, #16, #32,
    lora_dropout = 0.05,
    # init_lora_weights="gaussian",
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3711,
    use_rslora = False, #True, #False,
    loftq_config = None,
    max_seq_length = max_seq_length,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/494 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Unsloth 2024.11.10 patched 40 layers with 0 QKV layers, 0 O layers and 0 MLP layers.
Unsloth: Already have LoRA adapters! We shall skip this step.


In [12]:
from datasets import load_dataset, concatenate_datasets

# 指定フォルダ内のすべてのJSONファイルを読み込む
#INSTRUCT_FILES = "../instruction/*.json"
# INSTRUCT_FILES = "../instruction/ichikara-instruction-003-001-1.json"
# dataset = load_dataset("json", data_files=INSTRUCT_FILES)
# Ichikara データセット
INSTRUCT_FILES = "../instruction/ichikara-instruction-003-001-1.json"
ichikara_dataset = load_dataset("json", data_files=INSTRUCT_FILES)

Generating train split: 0 examples [00:00, ? examples/s]

In [13]:
# # Tengentoppa データセット（1万件のランダムサンプリング）
# tengentoppa_full = load_dataset("DeL-TaiseiOzaki/Tengentoppa-sft-v1.0")
# # シャッフルしてから最初の1万件を選択
# tengentoppa_dataset = tengentoppa_full["train"].shuffle(seed=42).select(range(10000))
# print(f"Tengentoppaデータセット: 元の{len(tengentoppa_full['train'])}件から{len(tengentoppa_dataset)}件をサンプリング")

In [14]:
# from datasets import load_dataset, Dataset
# import pandas as pd

# # Tengentoppa データセットをロード
# tengentoppa_full = load_dataset("DeL-TaiseiOzaki/Tengentoppa-sft-v1.0")

# #tengentoppa_dataset = tengentoppa_full["train"]
# # 先頭から1万件を選択
# tengentoppa_dataset = tengentoppa_full["train"].select(range(10000))
# # 10001番目から最後までのデータを選択
# start_index = 10000
# tengentoppa_remaining_dataset = tengentoppa_full["train"].select(range(start_index, len(tengentoppa_full["train"])))

# # データセット情報を出力
# print(f"Tengentoppaデータセット: 元の{len(tengentoppa_full['train'])}件から{len(tengentoppa_dataset)}件を選択")

# # データをPandas DataFrameに変換
# tengentoppa_df = pd.DataFrame(tengentoppa_dataset)

# # --- クレンジング処理 ---
# # 1. セル内の複数改行を削除、前後の空白をトリム
# tengentoppa_df = tengentoppa_df.applymap(
#     lambda x: x if not isinstance(x, str) else x.replace("\n\n", "").strip()
# )

# # 2. 空白または改行のみの行を削除
# tengentoppa_df = tengentoppa_df.loc[~tengentoppa_df.apply(
#     lambda row: row.astype(str).str.strip().eq('').all(), axis=1
# )]

# # クレンジング後のデータをHugging Face Dataset形式に戻す
# tengentoppa_dataset_cleaned = Dataset.from_pandas(tengentoppa_df)

# # データセット情報を出力
# print(f"クレンジング後のデータセット件数: {len(tengentoppa_dataset_cleaned)}件")


In [15]:
# ELYZA-tasks-100をダウンロードしてデータセットとして読み込み
from datasets import load_dataset

elyza_dataset = load_dataset("elyza/ELYZA-tasks-100")
elyza_dataset

README.md:   0%|          | 0.00/8.37k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/115k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['input', 'output', 'eval_aspect'],
        num_rows: 100
    })
})

In [19]:
prompt_template = """### 指示
{}
### 回答
{}"""

# フォーマット関数
def format_ichikara(examples):
    text = prompt_template.format(
        examples["text"],
        examples["output"]
    ) + tokenizer.eos_token
    return {"formatted_text": text}

def format_tengentoppa(examples):
    text = prompt_template.format(
        examples["instruction"],  # instructionフィールドを使用
        examples["output"]       # outputフィールドを使用
    ) + tokenizer.eos_token
    return {"formatted_text": text}

def format_elyza(examples):
    text = prompt_template.format(
        examples["input"],
        examples["output"]
    ) + tokenizer.eos_token
    return {"formatted_text": text}

# データセットのフォーマット
ichikara_formatted = ichikara_dataset.map(
    format_ichikara,
    num_proc=4,
)

# tengentoppa_formatted = tengentoppa_dataset_cleaned.map(
#     format_tengentoppa,
#     num_proc=4,
# )

elyza_formatted = elyza_dataset.map(
    format_elyza,
    num_proc=4,
)

# データセットの結合
combined_dataset = concatenate_datasets([
    # ichikara_formatted["train"],
    elyza_formatted["test"],
])

In [20]:
combined_dataset[2]

{'input': '古代ギリシャを学ぶ上で知っておくべきポイントは？',
 'output': '古代ギリシャは、古代文明の中で重要な役割を担った文化であり、西洋文明の原点とされています。主要なポイントとしては以下のような点が挙げられます。\n\n古代ギリシャには多くの島があり、それぞれが独自の政治・文化・経済を持っていた。\n古代ギリシャには多くの偉大な哲学者や文学者、芸術家、科学者などがいた。\nスパルタ、アテナイ、オリンピアなどの古代ギリシャの城邦があり、それぞれが独自の政治体制を持っていた。\n古代ギリシャは古代ローマによって征服され、ギリシャ文化はローマ文化に大きな影響を与えた。\n古代ギリシャは、古代オリンピック競技会などのスポーツ、古代神話、古代建築なども有名である。',
 'eval_aspect': '- 「独自の文化や哲学、神話が有名です」などのように具体例がない場合は-1点\n- 事実と異なる内容の場合: -2点\n',
 'formatted_text': '### 指示\n古代ギリシャを学ぶ上で知っておくべきポイントは？\n### 回答\n古代ギリシャは、古代文明の中で重要な役割を担った文化であり、西洋文明の原点とされています。主要なポイントとしては以下のような点が挙げられます。\n\n古代ギリシャには多くの島があり、それぞれが独自の政治・文化・経済を持っていた。\n古代ギリシャには多くの偉大な哲学者や文学者、芸術家、科学者などがいた。\nスパルタ、アテナイ、オリンピアなどの古代ギリシャの城邦があり、それぞれが独自の政治体制を持っていた。\n古代ギリシャは古代ローマによって征服され、ギリシャ文化はローマ文化に大きな影響を与えた。\n古代ギリシャは、古代オリンピック競技会などのスポーツ、古代神話、古代建築なども有名である。</s>'}

In [21]:
"""
training_arguments: 学習の設定

  - output_dir:
      -トレーニング後のモデルを保存するディレクトリ

  - per_device_train_batch_size:
      - デバイスごとのトレーニングバッチサイズ

  - per_device_eval_batch_size:
      - デバイスごとの評価バッチサイズ

  - gradient_accumulation_steps:
      - 勾配を更新する前にステップを積み重ねる回数

  - optim:
      - オプティマイザの設定

  - num_train_epochs:
      - エポック数

  - eval_strategy:
      - 評価の戦略 ("no"/"steps"/"epoch")

  - eval_steps:
      - eval_strategyが"steps"のとき、評価を行うstep間隔

  - logging_strategy:
      - ログ記録の戦略

  - logging_steps:
      - ログを出力するステップ間隔

  - warmup_steps:
      - 学習率のウォームアップステップ数

  - save_steps:
      - モデルを保存するステップ間隔

  - save_total_limit:
      - 保存しておくcheckpointの数

  - max_steps:
      - トレーニングの最大ステップ数

  - learning_rate:
      - 学習率

  - fp16:
      - 16bit浮動小数点の使用設定（第8回演習を参考にすると良いです）

  - bf16:
      - BFloat16の使用設定

  - group_by_length:
      -  入力シーケンスの長さによりバッチをグループ化 (トレーニングの効率化)

  - report_to:
      - ログの送信先 ("wandb"/"tensorboard"など)
"""
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset=combined_dataset, #dataset["train"],
    max_seq_length = max_seq_length,
    dataset_text_field="formatted_text",
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2, #8, #4, #2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 1,
        logging_steps = 10,
        warmup_steps = 10,
        save_steps=100,
        save_total_limit=2,
        max_steps=-1,
        # optim="adamw_torch_fused",
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        group_by_length=True,
        seed = 3711,
        output_dir = "outputs",
        report_to = "none",
        # report_to="wandb",
        # auto_find_batch_size=True, # 追加：これを入れないとGPUメモリがオーバーフローする
    ),
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [22]:
#@title 現在のメモリ使用量を表示
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
9.209 GB of memory reserved.


In [23]:
#@title 学習実行
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 100 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 12
 "-____-"     Number of trainable parameters = 125,173,760


Step,Training Loss
10,2.035000


In [24]:
# ELYZA-tasks-100-TVの読み込み。事前にファイルをアップロードしてください
# データセットの読み込み。
# omnicampusの開発環境では、左にタスクのjsonlをドラッグアンドドロップしてから実行。
import json
datasets = []
with open("../elyza-tasks-100-TV_0.jsonl", "r") as f:
    item = ""
    for line in f:
      line = line.strip()
      item += line
      if item.endswith("}"):
        datasets.append(json.loads(item))
        item = ""

In [ ]:
%%time
# 学習したモデルを用いてタスクを実行
from tqdm import tqdm
import time

start_time = time.time()  # 開始時間を記録

# 推論するためにモデルのモードを変更
FastLanguageModel.for_inference(model)

results = []
for dt in tqdm(datasets):
  input = dt["input"]

  prompt = f"""### 指示\n{input}\n### 回答\n"""

  inputs = tokenizer([prompt], return_tensors = "pt").to(model.device)

  outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True, do_sample=False, repetition_penalty=1.2)
  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).split('\n### 回答')[-1]

  results.append({"task_id": dt["task_id"], "input": input, "output": prediction})

# 推論にかかった時間を計算して表示
end_time = time.time()  # 終了時間を記録
total_time = end_time - start_time
print(f"推論にかかった時間: {total_time:.2f} 秒")

  4%|▍         | 4/100 [00:47<15:05,  9.43s/it]

In [ ]:
# jsonlで保存
with open(f"{new_model_id}_output.jsonl", 'w', encoding='utf-8') as f:
    for result in results:
        json.dump(result, f, ensure_ascii=False)
        f.write('\n')

In [ ]:
!pip install -qqq openai

In [ ]:
# シークレットキーを使う場合
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
%%time
import openai
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

# https://github.com/openai/openai-cookbook/blob/main/examples/How_to_handle_rate_limits.ipynb

client = openai.OpenAI(api_key=OPENAI_API_KEY)

@retry(wait=wait_random_exponential(min=2, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return client.chat.completions.create(**kwargs)


def gpt4eval_tv(input_text, output_text):
    prompt = f"""あなたは採点者です。

問題, 採点基準, 回答 が与えられます。

採点基準を参考にして、回答を1,2,3,4,5の5段階で採点し、数字のみを出力してください。

# 問題
{input_text}

# 採点基準
基本的な採点基準
- 1点: 誤っている、 指示に従えていない
- 2点: 誤っているが、方向性は合っている
- 3点: 部分的に誤っている、 部分的に合っている
- 4点: 合っている
- 5点: 役に立つ

基本的な減点項目
- 不自然な日本語: -1点
- 部分的に事実と異なる内容を述べている: -1点
- 「倫理的に答えられません」のように過度に安全性を気にしてしまっている: 2点にする

# 回答
{output_text}
"""

    # print(prompt) # for debug

    response = completion_with_backoff(
        # model="gpt-4o-2024-11-20",
        # model="gpt-4o",
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        frequency_penalty=0,
        presence_penalty=0,
    )
    # print(response.choices[0].message.content)
    gpt4score = response.choices[0].message.content
    try:
        gpt4score = int(gpt4score)
    except ValueError:
        gpt4score = None
    return gpt4score

In [ ]:
for result in tqdm(results):
    score = gpt4eval_tv(result["input"], result["output"])
    print(score)
    result["gpt4score"] = score

In [ ]:
# リストからgpt4scoreの平均を計算
gpt4score_average = sum(item['gpt4score'] for item in results) / len(results)

# 結果を出力
print(f'gpt4scoreの平均は: {gpt4score_average}')

In [ ]:
import pandas as pd

df = pd.DataFrame(results)

# CSVファイルとして保存
csv_file_path = f"{new_model_id}_tv.csv"
df.to_csv(csv_file_path, index=False, encoding='utf-8')

# エクセルファイルとして保存
excel_file_path = f"{new_model_id}_tv.xlsx"
df.to_excel(excel_file_path, index=False)

モデルとトークナイザーをHugging Faceにアップロードします。  
本コードではLoRAのアダブタのみを保存します。  
このアダプタを用いた推論方法はModel_Inference_Template_unsloth_20241127.ipynbをご参照ください。  
  
一旦privateでアップロードしてください。  
https://docs.unsloth.ai/basics/saving-and-using-models

In [ ]:
# LoRAアダプタだけ保存
model.push_to_hub_merged(
    new_model_id+"_lora",
    tokenizer=tokenizer,
    save_method="lora",
    token=HF_TOKEN,
    private=True
)

In [ ]:
# ELYZA-tasks-100をダウンロードしてデータセットとして読み込み
from datasets import load_dataset

datasets = load_dataset("elyza/ELYZA-tasks-100")

In [ ]:
# ELYZA-tasks-100 で推論する

import time
from tqdm import tqdm

# モデルを用いてタスクの推論。

# 推論するためにモデルのモードを変更
FastLanguageModel.for_inference(model)

results = []
start_time = time.time()  # 開始時間を記録

results = []
for dt in tqdm(datasets["test"]):
    input = dt["input"]
    prompt = f"""### 指示\n{input}\n### 回答\n"""

    inputs = tokenizer([prompt], return_tensors = "pt").to(model.device)

    # outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True, do_sample=False, repetition_penalty=1.2)
    outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True, do_sample=False, repetition_penalty=1.2)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).split('\n### 回答')[-1]

    # print(input)
    # print(prediction)

    results.append({"input": input, "output": dt["output"], "eval_aspect": dt["eval_aspect"], "prediction": prediction})


# 推論にかかった時間を計算して表示
end_time = time.time()  # 終了時間を記録
total_time = end_time - start_time
print(f"推論にかかった時間: {total_time:.2f} 秒")

In [ ]:
# !pip install openai

In [ ]:
# シークレットキーを使う場合
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
# GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [ ]:
%%time
import openai
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

# https://github.com/openai/openai-cookbook/blob/main/examples/How_to_handle_rate_limits.ipynb

client = openai.OpenAI(api_key=OPENAI_API_KEY)

@retry(wait=wait_random_exponential(min=2, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return client.chat.completions.create(**kwargs)


def gpt4eval(pred, input_text, output_text, eval_aspect):
    prompt = f"""あなたは採点者です。

問題, 正解例, 採点基準, 回答 が与えられます。

採点基準と正解例を参考にして、回答を1,2,3,4,5の5段階で採点し、数字のみを出力してください。

# 問題
{input_text}

# 正解例
{output_text}

# 採点基準
基本的な採点基準
- 1点: 誤っている、 指示に従えていない
- 2点: 誤っているが、方向性は合っている
- 3点: 部分的に誤っている、 部分的に合っている
- 4点: 合っている
- 5点: 役に立つ

基本的な減点項目
- 不自然な日本語: -1点
- 部分的に事実と異なる内容を述べている: -1点
- 「倫理的に答えられません」のように過度に安全性を気にしてしまっている: 2点にする

問題固有の採点基準
{eval_aspect}

# 回答
{pred}
"""

    # print(prompt) # for debug

    response = completion_with_backoff(
        # model="gpt-4o-2024-11-20",
        # model="gpt-4o",
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        frequency_penalty=0,
        presence_penalty=0,
    )
    # print(response.choices[0].message.content)
    gpt4score = response.choices[0].message.content
    try:
        gpt4score = int(gpt4score)
    except ValueError:
        gpt4score = None
    return gpt4score

In [ ]:
for result in tqdm(results):
    score = gpt4eval(result["prediction"], result["input"], result["output"], result["eval_aspect"])
    print(score)
    result["gpt4score"] = score

In [ ]:
# リストからgpt4scoreの平均を計算
gpt4score_average = sum(item['gpt4score'] for item in results) / len(results)

# 結果を出力
print(f'gpt4scoreの平均は: {gpt4score_average}')

In [ ]:
import pandas as pd

df = pd.DataFrame(results)

# CSVファイルとして保存
csv_file_path = f"{new_model_id}_output.csv"
df.to_csv(csv_file_path, index=False, encoding='utf-8')

# エクセルファイルとして保存
excel_file_path = f"{new_model_id}_output.xlsx"
df.to_excel(excel_file_path, index=False)

In [ ]:
# pandasの表を表示
from IPython.display import display
display(df)